# Derived Portfolios

In [29]:
import os
import json
from datetime import datetime, timedelta
import pytz

import lusid
import lusid.models as models
from lusid.exceptions import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

import pandas as pd

pd.set_option('display.max_columns', None)

secrets_path = '/Users/msingh/Projects/lusidws/secrets.json'

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name='LusidJupyterNotebook'
)

In [30]:
with open(r'config/build_transactions_mapping.json') as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)
    
with open(r'config/get_holdings_mapping.json') as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

## 1) Load default transactions into a new scope

In [31]:
scope = 'notebook-derived-portfolios'
portfolio_code = 'EQUITY-UK' + '-' + create_scope_id()
portfolio_code

'EQUITY-UK-3a46-f87c-1350-d6'

In [32]:
transactions_file = r'data/derived/equity_transactions.csv'
transactions_df = pd.read_csv(transactions_file)
transactions_df['portfolio_code'] = portfolio_code

In [33]:
seed_data_response = seed_data(
    api_factory,
    ['portfolios', 'instruments', 'transactions'],
    scope,
    transactions_df,
    'DataFrame',
)

In [34]:
transactions_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
derived_portfolios_api = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)

## 2) Lets check our holdings

In [35]:
response = transactions_portfolio_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=['Instrument/default/Name']
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)
holdings_df

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
0,default,LUID_00003D60,{},Aviva,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,132000.0,132000.0,660000.0,GBP,660000.0,GBP,GBP
1,default,LUID_00003D61,{},BHP,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,120000.0,120000.0,2160000.0,GBP,2160000.0,GBP,GBP
2,default,LUID_00003D68,{},Barclays,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,300000.0,300000.0,600000.0,GBP,600000.0,GBP,GBP
3,default,LUID_00003D67,{},BP,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,200000.0,200000.0,1000000.0,GBP,1000000.0,GBP,GBP
4,default,LUID_00003D62,{},HSBC,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,40000.0,40000.0,240000.0,GBP,240000.0,GBP,GBP
5,default,CCY_GBP,{},GBP,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,B,3260000.0,3260000.0,3260000.0,GBP,3260000.0,GBP,GBP
6,default,LUID_00003D63,{},Morrisons,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,360000.0,360000.0,720000.0,GBP,720000.0,GBP,GBP
7,default,LUID_00003D5Y,{},Tesco,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,12000.0,12000.0,100000.0,GBP,100000.0,GBP,GBP
8,default,LUID_00003D65,{},Rightmove,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,160000.0,160000.0,960000.0,GBP,960000.0,GBP,GBP
9,default,LUID_00003D5Z,{},vodafone,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,900000.0,900000.0,900000.0,GBP,900000.0,GBP,GBP


## 3) What transactions make up our Barclays holdings?

In [36]:
build_transactions_response = transactions_portfolio_api.build_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_query_parameters=models.TransactionQueryParameters(
        start_date='2020-01-01', end_date='2021-12-31'
    ),
    property_keys=['Instrument/default/Name']
)

build_transactions_df = lusid_response_to_data_frame(
    build_transactions_response,
    rename_properties=True,
    column_name_mapping=build_transactions_json_mapping,
)

build_transactions_df.query("InstrumentName == 'Barclays'")

,TransactionId,TransactionType,TransactionTypeDesc,ClientId,instrument_scope,LusidInstrumentId,TransactionDate,SettlementDate,Units,TransactionAmount,Price,PriceType,TotalConsideration,TotalConsiderationCurrency,ExchangeRate,TransactionToPortfolioRate,TransactionCurrency,strategy(notebook-derived-portfolios-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),ResultantHolding,InstrumentName,TransactionStatus,EntryDateTime,RealisedGainLoss,InstrumentCurrency
4,trd_0005,Buy,Purchase,EQ_1236,default,LUID_00003D68,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,150000.0,Barclays,Active,2021-12-24 15:02:55.463113+00:00,[],NaN
5,trd_0006,Buy,Purchase,EQ_1236,default,LUID_00003D68,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,300000.0,Barclays,Active,2021-12-24 15:02:55.463113+00:00,[],NaN


## 4) Create a derived portfolio

In [37]:
new_scope = 'TempReporting' + '-' + scope
print(f'The scope we will use for the derived portfolios: {new_scope}')

The scope we will use for the derived portfolios: TempReporting-notebook-derived-portfolios


In [38]:
try:
    derived_portfolios_api.create_derived_portfolio(scope=new_scope, 
        create_derived_transaction_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
            display_name=portfolio_code,
            description='Reporting Portfolio',
            code=portfolio_code,
            parent_portfolio_id=models.ResourceId(scope=scope, code=portfolio_code),
            created='2020-01-01',
            corporate_action_source_id=None,
            accounting_method=None,
            sub_holding_keys=None
        ))

except ApiException as e:
    print(json.loads(e.body)['name'])
    print(json.loads(e.body)['title'])

In [39]:
response = transactions_portfolio_api.get_transactions(scope=new_scope, code=portfolio_code)
transactions_df = lusid_response_to_data_frame(
    response,
    rename_properties=True,
    column_name_mapping=build_transactions_json_mapping
)
transactions_df

,TransactionId,TransactionType,ClientId,instrument_scope,LusidInstrumentId,TransactionDate,SettlementDate,Units,Price,PriceType,TotalConsideration,TotalConsiderationCurrency,ExchangeRate,TransactionCurrency,strategy(notebook-derived-portfolios-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),EntryDateTime,InstrumentCurrency
0,trd_0001,Buy,EQ_1234,default,LUID_00003D60,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,120000.0,5.0,Price,600000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
1,trd_0002,Buy,EQ_1234,default,LUID_00003D60,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,12000.0,5.0,Price,60000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
2,trd_0003,Buy,EQ_1235,default,LUID_00003D61,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,60000.0,18.0,Price,1080000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
3,trd_0004,Buy,EQ_1235,default,LUID_00003D61,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,60000.0,18.0,Price,1080000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
4,trd_0005,Buy,EQ_1236,default,LUID_00003D68,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,2.0,Price,300000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
5,trd_0006,Buy,EQ_1236,default,LUID_00003D68,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,2.0,Price,300000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
6,trd_0007,Buy,EQ_1237,default,LUID_00003D67,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,100000.0,5.0,Price,500000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
7,trd_0008,Buy,EQ_1237,default,LUID_00003D67,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,100000.0,5.0,Price,500000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
8,trd_0009,Buy,EQ_1238,default,LUID_00003D62,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,20000.0,6.0,Price,120000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN
9,trd_0010,Buy,EQ_1238,default,LUID_00003D62,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,20000.0,6.0,Price,120000.0,GBP,1.0,GBP,ftse_tracker,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,2021-12-24 15:02:55.463113+00:00,NaN


In [40]:
cancel_response = transactions_portfolio_api.cancel_transactions(
    scope=new_scope, code=portfolio_code, transaction_ids=['trd_0006']
)

## 6) Check holdings on the primary and the derived portfolio

In [43]:
response = transactions_portfolio_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=['Instrument/default/Name']
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df.query("InstrumentName == 'Barclays'")

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
2,default,LUID_00003D68,{},Barclays,EQUITY-UK-3a46-f87c-1350-d6,notebook-derived-portfolios,P,300000.0,300000.0,600000.0,GBP,600000.0,GBP,GBP


In [44]:
response = transactions_portfolio_api.get_holdings(
    scope=new_scope, code=portfolio_code, property_keys=['Instrument/default/Name']
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df.query("InstrumentName == 'Barclays'")

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
2,default,LUID_00003D68,{},Barclays,EQUITY-UK-3a46-f87c-1350-d6,TempReporting-notebook-derived-portfolios,P,150000.0,150000.0,300000.0,GBP,300000.0,GBP,GBP
